In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
import numpy as np
import os
import glob
from collections import Counter, defaultdict

train_folder = "flags/train"
test_folder = "flags/test"

train_images = []
train_labels = []
test_images = []
test_labels = []

def load_images_from_folder(folder, images_list, labels_list, file_paths_list):
    target_size = (64, 64)
    for country_folder in os.listdir(folder):
        country_path = os.path.join(folder, country_folder)
        if os.path.isdir(country_path):
            for img_path in glob.glob(os.path.join(country_path, "*.png")):
                try:
                    img = Image.open(img_path).convert("RGB").resize(target_size)
                    img_array = np.array(img) / 255.0
                    images_list.append(img_array.flatten())
                    label = country_folder
                    labels_list.append(label)
                    file_paths_list.append(img_path)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

train_file_paths = []
test_file_paths = []
load_images_from_folder(train_folder, train_images, train_labels, train_file_paths)
load_images_from_folder(test_folder, test_images, test_labels, test_file_paths)

X_train = np.array(train_images)
y_train = np.array(train_labels)
X_test = np.array(test_images)
y_test = np.array(test_labels)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Model Accuracy:", accuracy)
print("Classification Report:\n", report)

prediction_details = defaultdict(lambda: defaultdict(list))
for true_label, predicted_label, file_path in zip(y_test, y_pred, test_file_paths):
    prediction_details[true_label][predicted_label].append(file_path)

with open("predictions_rf.txt", "w") as f:
    f.write(f"Model Accuracy: {accuracy}\n")
    f.write("Classification Report:\n")
    f.write(report)
    for true_label, counter in prediction_details.items():
        f.write(f"\nTrue Label: {true_label}\n")
        for predicted_label, file_paths in counter.items():
            file_names = [os.path.basename(path) for path in file_paths]
            f.write(f"  {', '.join(file_names)} are predicted as {predicted_label}: {len(file_paths)} times\n")

print("Predictions have been written to predictions_rf.txt")